# Import

In [1]:
import pandas as pd
import glob
import os
import numpy as np
import re
import unicodedata
import unidecode
import requests
import datetime
import csv
import pandas as pd
import streamlit as st
import plotly.graph_objects as go
from datetime import datetime
from currency_converter import CurrencyConverter

from unicodedata import normalize

In [2]:
import os, re, glob, csv
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

# Concaténation data

In [77]:
INPUT_DIR = "data"
OUTPUT_CSV = "out/all_rows.csv"
CHUNKSIZE = 20_000

def normalize_col(c: str) -> str:
    if c is None: return ""
    c2 = c.strip().lower()
    return {"date":"date", "negociant":"negociant"}.get(c2, c2)

def extract_negociant(path: str) -> str:
    base = Path(path).name
    m = re.match(r"^\d{8}_(.+?)\.csv$", base)
    return m.group(1) if m else os.path.splitext(base)[0]

def extract_date_from_filename(path: str) -> str | None:
    m = re.search(r"(\d{4})(\d{2})(\d{2})_", Path(path).name)
    return f"{m.group(1)}-{m.group(2)}-{m.group(3)}" if m else None

def read_csv_chunked(file: str, chunksize: int):
    kw = dict(sep=None, engine="python", chunksize=chunksize, dtype=str, on_bad_lines="warn")
    try:
        return pd.read_csv(file, encoding="latin-1", encoding_errors="replace", **kw)
    except TypeError:
        return pd.read_csv(file, encoding="latin-1", **kw)

csv_files = sorted(glob.glob(os.path.join(INPUT_DIR, "*.csv")))
if not csv_files:
    raise SystemExit(f"Aucun CSV dans {INPUT_DIR}")

# Découverte colonnes
all_cols = set()
for f in tqdm(csv_files, desc="Découverte colonnes"):
    try:
        cols = pd.read_csv(f, nrows=0, sep=None, engine="python", encoding="latin-1").columns
    except Exception:
        try:
            cols = pd.read_csv(f, nrows=0, sep=None, engine="python", encoding="utf-8-sig").columns
        except Exception:
            cols = []
    all_cols.update(normalize_col(c) for c in cols)

all_cols.update(["date","negociant"])
TARGET_COLS = ["date","negociant"] + sorted(c for c in all_cols if c not in ["date","negociant"])

os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
# (ré)écrit l’en-tête une seule fois
pd.DataFrame(columns=TARGET_COLS).to_csv(OUTPUT_CSV, index=False, lineterminator="\n")

tot_in = tot_out = 0
for file in tqdm(csv_files, desc="Ingestion"):
    negociant = extract_negociant(file)
    date_from_name = extract_date_from_filename(file)

    try:
        for chunk in read_csv_chunked(file, CHUNKSIZE):
            tot_in += len(chunk)
            chunk.columns = [normalize_col(c) for c in chunk.columns]
            if "negociant" not in chunk.columns:
                chunk["negociant"] = negociant
            if "date" not in chunk.columns:
                chunk["date"] = date_from_name
            else:
                if date_from_name is not None:
                    mask_empty = chunk["date"].isna() | (chunk["date"].astype(str).str.strip() == "")
                    chunk.loc[mask_empty, "date"] = date_from_name

            chunk = chunk.reindex(columns=TARGET_COLS).fillna("")
            chunk.to_csv(OUTPUT_CSV, mode="a", index=False, header=False, lineterminator="\n")
            tot_out += len(chunk)
    except Exception as e:
        print(f"[ERROR] Lecture impossible: {file} -> {e}")
        continue

print(f"Terminé. Lignes lues: {tot_in}, lignes écrites: {tot_out}")
print(f"CSV: {OUTPUT_CSV}")


Ingestion: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14020/14020 [09:28<00:00, 24.68it/s]

Terminé. Lignes lues: 51087394, lignes écrites: 51087394
CSV: out/all_rows.csv


In [3]:
import duckdb

IN  = "out/all_rows.csv"
OUT = "out/lynch_talbot_rows.csv"

con = duckdb.connect()
con.execute(f"""
  COPY (
    SELECT *
    FROM read_csv_auto('{IN}', sample_size=-1, nullstr='', ALL_VARCHAR=TRUE)
    WHERE
      (
        COALESCE(chateau, '')    ILIKE '%lynch%bages%'
     OR COALESCE(vin, '')        ILIKE '%lynch%bages%'
     OR COALESCE(producteur, '') ILIKE '%lynch%bages%'
      )
      OR
      (
        COALESCE(chateau, '')    ILIKE '%talbot%'
     OR COALESCE(vin, '')        ILIKE '%talbot%'
     OR COALESCE(producteur, '') ILIKE '%talbot%'
      )
  )
  TO '{OUT}' (HEADER, DELIMITER ',');
""")

# Compter
n = con.execute(f"SELECT COUNT(*) FROM read_csv_auto('{OUT}', sample_size=-1, ALL_VARCHAR=TRUE)").fetchone()[0]
print("Lignes extraites:", n)

con.close()



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Lignes extraites: 388611


# Récupération et cleaning data

In [5]:
general_df = pd.read_csv(OUT)

/tmp/ipykernel_421379/941311877.py:1: DtypeWarning: Columns (12,19) have mixed types. Specify dtype option on import or set low_memory=False.
  general_df = pd.read_csv(OUT)


In [7]:
general_df.vin = general_df['vin'].fillna(general_df['chateau']).fillna(general_df['producteur'])
general_df = general_df.drop(columns=["prix","pays","price","classification","source"])

In [6]:
# general_df.to_csv("general_df.csv",index=None)

general_df  = pd.read_csv("general_df.csv",encoding="latin-1")

/tmp/ipykernel_423308/3872190230.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  general_df  = pd.read_csv("general_df.csv",encoding="latin-1")


In [7]:
general_df = general_df.drop(columns=['details', 'detail'], errors='ignore')

general_df['prix_unitaire'] = general_df['prix_unitaire'].fillna(general_df['price_per_unit'])

general_df = general_df.drop(columns=['price_per_unit'], errors='ignore')

In [8]:
general_df = general_df.drop(columns=["lwin","conditionnement"])

In [9]:

# 2) Haystack = Series (pas DataFrame)
haystack = general_df['vin'].fillna('').astype(str)  # Series


# 4) Exclusion
exclude_terms = ["echo", "moussas","blanc","Averous", "berry", "3rd","ConnÃ©table","connetable","caillou","case"]
if exclude_terms:
    exclude_pattern = re.compile("|".join(map(re.escape, exclude_terms)), re.I)
    mask_exclude = haystack.str.contains(exclude_pattern, na=False)
else:
    mask_exclude = pd.Series(False, index=general_df.index)

# 5) Filtrage
sub_df = general_df[~mask_exclude].copy()




In [10]:
def simplify_couleur(couleur):
    couleur = couleur.lower()
    if "red" in couleur or "rouge" in couleur:
        return "red"
    elif "white" in couleur or "blanc" in couleur or "dry" in couleur or "sweet" in couleur:
        return "white"
    elif "rose" in couleur or "rosé" in couleur:
        return "rose"
    else:
        return couleur

sub_df['couleur'] = sub_df['couleur'].apply(simplify_couleur)

In [11]:
df = sub_df

In [12]:
format_mapping = {
    '75cl':    ['75cl','BT','750ml',750.0,'BT6','BT9','BT4','BT1'," 75cl"],
    '150cl':   ['150cl','1.5L','MG','Magnum','magnum',1500.0,'MG1','MG3'," 150cl"],
    '37.5cl':  ['37.5cl','375ml',375.0,'Half bottle','Half Bottle','37cl','375cl',"HV"],
    '300cl':   ['300cl','DM','3000.0',3000.0,'Double Magnums','DM3','3L'],
    '600cl':   ['6L','6000.0',6000.0,'600cl','Imperial','IM'],
    '225cl':   ['MJ'],
    '900cl':   ['900cl','9L',9000.0,'SZ'],
    '1800cl':  ['1800cl','18L',18000.0,'MC','Melchior','melchior'],
    '1200cl':  ['12L','12000.0','Balthazar',12000.0],
    '1500cl':  ['1500cl','15L',15000.0,'Nabuchodonosor','NB','Nebuchadnezzar',
                'nabuchodonosor','nebuchadnezzar'],
    '2700cl':  ['2700cl','27L',27000.0,'PR','Primat','primat'],
    '500cl':   ['500cl','5L',5000.0,'JR','Jeroboam'],
    '50cl':    ['50cl',500.0],
    '450cl':   ['450cl','4.5L'],
    '10cl':    ['10cl'],
}

# --- 3) On compile un regex par clé
patterns = {}
for target, aliases in format_mapping.items():
    alt = [re.escape(str(a).strip().lower()) for a in aliases]
    # on trie par longueur décroissante pour capturer d'abord le plus long
    alt.sort(key=len, reverse=True)
    patterns[target] = re.compile(r'(' + '|'.join(alt) + r')', re.I)

# --- 4) Fonction de mapping
def map_format(raw):
    s = str(raw).strip().lower().replace('×','x')
    for target, pat in patterns.items():
        if pat.search(s):
            return target
    return np.nan

# --- 5) On l'applique
df['format'] = df['format'].apply(map_format)


In [13]:
df = df[~df.quantite.isna()]
df = df[~df.prix_unitaire.isna()]
df = df[~df.format.isna()]
df = df[df.couleur =="red"]

In [14]:
df['date_parsed'] = pd.to_datetime(
    df['date'].astype(str),
    dayfirst=True,
    infer_datetime_format=True,
    errors='coerce'
)

# 2a) Si vous voulez garder un type datetime64[ns]
df['date'] = df['date_parsed']

# 2b) Si vous préférez une string ISO 'YYYY-MM-DD'
df['date'] = df['date_parsed'].dt.strftime('%Y%m%d')

# 3) Optionnel : on supprime la colonne temporaire
df.drop(columns='date_parsed', inplace=True)


/tmp/ipykernel_423308/782755193.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date_parsed'] = pd.to_datetime(


# Final df à enregistrer + standardiser nom 

In [15]:
cond_talbot = df["vin"].str.contains(r"\bTalbot\b", case=False, na=False)
cond_lynch  = df["vin"].str.contains(r"\bLynch\b",  case=False, na=False)

df["vin_clean"] = np.select(
    [cond_talbot,          cond_lynch],
    ["Château Talbot",     "Château Lynch-Bages"],  # (typo corrigée : "Château")
    default=df["vin"]
)


In [16]:
df.drop(columns=["producteur","chateau"],inplace=True)

In [17]:
df = df[df.prix_unitaire>10]


In [18]:
df = df[df.negociant != "wine_bourse"]

df = df[df.negociant != "vinum_fine_wines"]

In [21]:
df.vin = df.vin_clean

In [22]:
df.to_csv('sub_df.csv',index=False)